# groupby 메서드로 평균값 구하기

In [3]:
# 집계 = 수집한 데이터를 바탕으로 의미 있는 통계량을 도출하는 것
import pandas as pd
df = pd.read_csv('../data/gapminder.tsv', sep='\t')

avg_life_exp_by_year = df.groupby('year')['lifeExp'].mean()
print(avg_life_exp_by_year)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


# 분할-반영-결합 과정 살펴보기

In [8]:
# groupby 메서드에 열 이름을 전달하는 행위가 '분할'
# 아래의 코드와 비슷한 연산을 수행한다고 이해하면 돼
years = df.year.unique()
print(years)

# 각 연도별로 데이터를 추출해서 평균값을 구하는 행위가 '반영'
# 마찬가지로 아래와 비슷한 연산인 셈
y1952 = df.loc[df.year == 1952, :]
print(y1952.head())

# lifeExp 열의 평균값을 구하는 행위도 '반영'
y1952_mean = y1952.lifeExp.mean()
print(y1952_mean)

# 마지막으로 연도별 lifeExp 평균값을 합치는 과정이 '결합'
# df2 = pd.DataFrame({"year":[1952, 1957, 1962, 2007],"":[y1952_mean, y1957_mean, y1962_mean, y2007_mean]})

[1952 1957 1962 1967 1972 1977 1982 1987 1992 1997 2002 2007]
        country continent  year  lifeExp       pop    gdpPercap
0   Afghanistan      Asia  1952   28.801   8425333   779.445314
12      Albania    Europe  1952   55.230   1282697  1601.056136
24      Algeria    Africa  1952   43.077   9279525  2449.008185
36       Angola    Africa  1952   30.015   4232095  3520.610273
48    Argentina  Americas  1952   62.485  17876956  5911.315053
49.057619718309866


# 평균값을 구하는 사용자 함수와 groupby 메서드

In [9]:
# agg 메서드 = 사용자 함수와 groupby 메서드를 조합하는 메서드
def my_mean(values):
    n = len(values)  
    sum = 0 
    for value in values: 
        sum += value 
    return sum / n 

# agg 메서드로 조합한 결과가 mean 메서드의 결과와 동일함
agg_my_mean = df.groupby('year').lifeExp.agg(my_mean)
print(agg_my_mean)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


# 2개의 인잣값을 받아 처리하는 사용자 함수와 groupby 메서드

In [11]:
# 첫 번째 인자로 받은 열의 평균값을 구하여
# 두 번째 인자로 받은 값과의 차이를 리턴
def my_mean_diff(values, diff_value):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    mean = sum / n
    return mean - diff_value

global_mean = df.lifeExp.mean()
print(global_mean)

agg_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff, diff_value=global_mean)
print(agg_mean_diff)

59.474439366197174
year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64


# 집계 메서드를 리스트, 딕셔너리에 담아 전달하기

In [13]:
# 여러 개의 집계 메서드를 한 번에 사용하고 싶을때 리스트, 딕셔너리로 agg 메서드에 전달
import numpy as np

# lifeExp 열의 0이 아닌 값의 개수, 평균, 표준편차를 한 번에 계산한 것
# np 메서드인 count_nonzero, mean, std
gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])
print(gdf)

# 딕셔너리_ 키에 열 이름, 값으로 집계 메서드를 전달
gdf_dict = df.groupby('year').agg({
    'lifeExp': 'mean',
    'pop': 'median',
    'gdpPercap': 'median'
})
print(gdf_dict)

      count_nonzero       mean        std
year                                     
1952            142  49.057620  12.225956
1957            142  51.507401  12.231286
1962            142  53.609249  12.097245
1967            142  55.678290  11.718858
1972            142  57.647386  11.381953
1977            142  59.570157  11.227229
1982            142  61.533197  10.770618
1987            142  63.212613  10.556285
1992            142  64.160338  11.227380
1997            142  65.014676  11.559439
2002            142  65.694923  12.279823
2007            142  67.007423  12.073021
        lifeExp         pop    gdpPercap
year                                    
1952  49.057620   3943953.0  1968.528344
1957  51.507401   4282942.0  2173.220291
1962  53.609249   4686039.5  2335.439533
1967  55.678290   5170175.5  2678.334740
1972  57.647386   5877996.5  3339.129407
1977  59.570157   6404036.5  3798.609244
1982  61.533197   7007320.0  4216.228428
1987  63.212613   7774861.5  4280.300366
19

C:\Users\njh45\AppData\Local\Temp\ipykernel_21756\3329073502.py:6: FutureWarning: The provided callable <function mean at 0x00000288D8675260> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])
C:\Users\njh45\AppData\Local\Temp\ipykernel_21756\3329073502.py:6: FutureWarning: The provided callable <function std at 0x00000288D86753A0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])


지금까진 집계 메서드. 이번에는 데이터 변환 메서드에 대해 학습

# 표준점수(Z점수) 계산하기

In [17]:
# 표준점수 = 데이터의 평균 - 표준편차
def my_zscore(x):
    return(x - x.mean()) / x.std()

# transform 메서드가 '변환' 메서드
transform_z = df.groupby('year').lifeExp.transform(my_zscore)
print(transform_z.head())

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64


# 누락값을 평균값으로 처리하기

In [19]:
import seaborn as sns
import numpy as np
np.random.seed(42)

tips_10 = sns.load_dataset('tips').sample(10)
# .random.permutation(열.index)_ 선택된 열의 인덱스를 무작위로 섞음
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NaN
print(tips_10)

# total_bill 열에는 남/여 데이터가 섞여있으니 구별해야 데이터 훼손이 안돼
count_sex = tips_10.groupby('sex').count()
print(count_sex)

# 성별을 구분하여 평균값을 구하는 함수
def fill_na_mena(x):
    avg = x.mean()
    return x.fillna(avg) # fillna 메서드는 NaN을 지정된 값으로 대체

# total_bill을 위 함수에 전달하여 return 받고 tips_10에 새로운 열로 추가한 것
# 남/여의 누락값을 고려하여 계산한 평균값으로 잘 채워져 있음
total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)
tips_10['fill_total_bill'] = total_bill_group_mean
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153         NaN  2.00    Male     No   Sun  Dinner     4
211         NaN  5.16    Male    Yes   Sat  Dinner     4
198         NaN  2.00  Female    Yes  Thur   Lunch     2
176         NaN  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2
        total_bill  tip  smoker  day  time  size
sex                                             
Male             4    7       7    7     7     7
Female           2    3       3    3     3     3


C:\Users\njh45\AppData\Local\Temp\ipykernel_21756\553174750.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  count_sex = tips_10.groupby('sex').count()


# 데이터 필터링 사용하기 ─ filter 메서드

In [28]:
tips = sns.load_dataset('tips')
print(tips.shape)

print(tips['size'].value_counts())

# 1,5,6 주문이 너무 적으니까 제외하고 그룹화
tips_filtered = tips.groupby('size').filter(
    lambda x : x['size'].count() >= 30
)
print(tips_filtered.shape)
print(tips_filtered['size'].value_counts())

(244, 7)
size
2    156
3     38
4     37
5      5
1      4
6      4
Name: count, dtype: int64
(231, 7)
size
2    156
3     38
4     37
Name: count, dtype: int64


# 그룹 오브젝트 저장하여 살펴보기

In [34]:
tips_10 = sns.load_dataset('tips').sample(10, random_state=42)
print(tips_10)

# groupby 해보면 type이 df GroupBy object
grouped = tips_10.groupby('sex')
print(grouped)

# 그룹 오브젝트에 포함된 그룹을 보기 위해 groups 속성 사용
# sex 열로 그룹화한 df의 인덱스를 확인할 수 있어
# 이걸 바탕으로 집계, 변환, 필터 작업을 수행하는거지
print(grouped.groups)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153       24.55  2.00    Male     No   Sun  Dinner     4
211       25.89  5.16    Male    Yes   Sat  Dinner     4
198       13.00  2.00  Female    Yes  Thur   Lunch     2
176       17.89  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2
{'Male': [24, 6, 153, 211, 176, 192, 9], 'Female': [198, 124, 101]}


C:\Users\njh45\AppData\Local\Temp\ipykernel_21756\3585191582.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = tips_10.groupby('sex')


# 그룹 오브젝트의 평균 구하기

In [41]:
# 평균값을 구할 수 없는 열이 있어도 자동으로 계산 가능한 열만 해줌
# avgs = grouped.mean()
# print(avgs)

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')


# 그룹 오브젝트에서 데이터 추출하고 반복하기

In [64]:
# 특정 데이터만 추출하려면 get_group 메서드 사용
female = grouped.get_group('Female')
print(female)

# sex 열을 기준으로 그룹화된 tips 데이터 집합에 반복문을 사용
for sex_group in grouped:
    print(sex_group)

     total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2
('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2)
('Female',      total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2)


# 여러 열을 사용하여 그룹 오브젝트 계산

In [ ]:
'''bill_sex_time = tips_10.groupby(['sex', 'time'])
group_avg = bill_sex_time.mean()
print(group_avg)

# group_avg 인덱스 출력
print(group_avg.index)

# 인덱스가 MultiIndex인 경우에는 reset_index 메서드로 새로 부여
group_method = tips_10.groupby(['sex', 'time']).mean().reset_index()
'''